In [1]:
import pandas as pd
import unicodedata
import numpy as np

# Función: Estandarización de texto (Mayúsculas y eliminación de tildes)
def limpiar_texto(s):
    """Limpia el texto, crucial para crear llaves de unión robustas."""
    if isinstance(s, str):
        s = s.strip().upper()
        # Normaliza a NFD y elimina tildes/caracteres diacríticos
        s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
        return s
    return s

In [2]:
# Cargar DataFrames
df_properties = pd.read_csv('properties.csv')
df_barrios_ref = pd.read_csv('barrios_bogota_localidades_limpio2.csv')

# 2.1 Estandarización de la llave de unión
df_properties['barrio_limpio'] = df_properties['barrio'].apply(limpiar_texto)
df_barrios_ref['barrio_limpio'] = df_barrios_ref['Barrio'].apply(limpiar_texto)

# 2.2 Preparar referencia (eliminar duplicados por si un barrio pertenece a múltiples UPZ)
columnas_ref = ['Localidad', 'UPZ', 'barrio_limpio']
df_barrios_ref_unique = df_barrios_ref[columnas_ref].drop_duplicates(subset=['barrio_limpio'])

In [3]:
# 3.1 Diccionario de Corrección Manual (Consolidación de las dos rondas)
mapeo_correccion = {
    'SUBA': 'SUBA', 'CASTILLAS': 'KENNEDY', 'EL PRADO': 'SUBA', 'EL TINTAL': 'KENNEDY',
    'TIBABUYES': 'SUBA', 'CIUDAD USME': 'USME', 'LA ALHAMBRA': 'SUBA', 'USAQUEN': 'USAQUEN',
    'ENGATIVA': 'ENGATIVA', 'BOSA OCCIDENTAL': 'BOSA', 'BOSA CENTRAL': 'BOSA',
    'EL RINCON': 'SUBA', 'VERBENAL': 'USAQUEN', 'FONTIBON SAN PABLO': 'FONTIBON',
    'TOBERIN': 'USAQUEN', 'AMERICAS': 'KENNEDY', 'CHICO LAGO': 'CHAPINERO',
    'FONTIBON': 'FONTIBON', 'SOSIEGO': 'SAN CRISTOBAL', 'TINTAL NORTE': 'KENNEDY',
    'CASTILLA': 'KENNEDY', 'TINTAL SUR': 'KENNEDY', 'APOGEO': 'BOSA',
    'ARBORIZADORA': 'CIUDAD BOLIVAR', 'BOYACA REAL': 'ENGATIVA', 'MODELIA': 'FONTIBON',
    'LA FLORESTA': 'SUBA', 'CHAPINERO': 'CHAPINERO', 'CAPELLANIA': 'FONTIBON',
    'BOSA 61': 'BOSA', 'COMUNEROS': 'SUBA', 'CIUDAD SALITRE ORIENTAL': 'TEUSAQUILLO',
    'PATIO BONITO': 'KENNEDY', 'LOS ALCAZARES': 'BARRIOS UNIDOS'
}

# 3.2 Left Merge para asignar Localidad
df_merged = df_properties.merge(
    df_barrios_ref_unique,
    on='barrio_limpio',
    how='left'
)

# 3.3 Aplicar corrección manual a los NaN residuales
# Se utiliza .fillna() después de aplicar el mapeo a la columna limpia.
df_merged['Localidad'] = df_merged['Localidad'].fillna(df_merged['barrio_limpio'].map(mapeo_correccion))

# 3.4 Limpieza final de la columna Localidad (p. ej., 'Usaquén' -> 'USAQUEN')
df_merged['Localidad'] = df_merged['Localidad'].apply(limpiar_texto)

# 3.5 Imputación de los residuales finales (los que no coincidieron ni en el merge ni en el mapeo)
df_merged['Localidad'] = df_merged['Localidad'].fillna('DESCONOCIDO')
df_merged['Localidad'] = df_merged['Localidad'].replace('NAN', 'DESCONOCIDO') # Por si alguna localidad limpia es 'NAN'

print("✅ Ingeniería de Características Geográficas completada. Nueva columna 'Localidad' creada.")

✅ Ingeniería de Características Geográficas completada. Nueva columna 'Localidad' creada.


In [4]:
# 4.1 Imprimir conteo de valores (validación)
conteo_final = df_merged['Localidad'].value_counts()

print("\n--- Feature Validation (Localidad) ---")
print(f"Número total de propiedades: {len(df_merged)}")
print(f"Total de categorías únicas de Localidad: {len(conteo_final)}")
print(f"Conteo de Localidades Desconocidas: {conteo_final.get('DESCONOCIDO', 0)}")
print("-" * 35)
print("Distribución de Localidades (Top 5):")
print(conteo_final.head(20))

# La columna 'Localidad' está lista para el One-Hot Encoding.


--- Feature Validation (Localidad) ---
Número total de propiedades: 585
Total de categorías únicas de Localidad: 16
Conteo de Localidades Desconocidas: 0
-----------------------------------
Distribución de Localidades (Top 5):
Localidad
SUBA                  148
KENNEDY               117
USAQUEN               108
USME                   46
ENGATIVA               44
FONTIBON               29
BOSA                   28
CIUDAD BOLIVAR         22
CHAPINERO              18
TEUSAQUILLO             5
PUENTE ARANDA           5
SAN CRISTOBAL           5
RAFAEL URIBE URIBE      4
BARRIOS UNIDOS          3
LOS MARTIRES            2
ANTONIO NARINO          1
Name: count, dtype: int64


In [5]:
df_merged.head()

,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,gas,parqueadero,precio,direccion,nombre,descripcion,barrio,barrio_limpio,Localidad,UPZ
0,Santa Mónica,532000,4.0,37,Si,86.0,1,1,1,1,...,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO,PARDO RUBIO,CHAPINERO,Pardo Rubio
1,Chicó Milano 101,0,6.0,7,Si,77.0,1,2,2,1,...,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN,USAQUEN,USAQUEN,NaN
2,Portal del Belmira,811893,4.0,14,Si,109.0,3,4,2,1,...,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS,LOS CEDROS,USAQUEN,Los Cedros
3,Carmel Reservado,400200,4.0,11,Si,76.0,3,2,1,2,...,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO,EL PRADO,SUBA,NaN
4,Toscana 1,270000,4.0,20,Si,105.0,4,2,1,0,...,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN,TOBERIN,USAQUEN,NaN


In [6]:

columnas_a_eliminar = ['conjunto', 'descripcion', 'UPZ','nombre','barrio']

#borrar columnas no necesarias
df_merged = df_merged.drop(
    columns=[col for col in columnas_a_eliminar if col in df_merged.columns]
)

In [7]:
df_merged.head()

,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,tipo_de_inmueble,deposito,porteria,zona_de_lavanderia,gas,parqueadero,precio,direccion,barrio_limpio,Localidad
0,532000,4.0,37,Si,86.0,1,1,1,1,Apartamento,0,24 hrs,No Tiene,Si,No,313900000.0,Avenida Carrera 3 # 59-65,PARDO RUBIO,CHAPINERO
1,0,6.0,7,Si,77.0,1,2,2,1,Apartamento,1,24 hrs,No Tiene,Si,Si,440100000.0,Carrera 12 #101A-18,USAQUEN,USAQUEN
2,811893,4.0,14,Si,109.0,3,4,2,1,Apartamento,0,24 hrs,No Tiene,Si,Si,495000000.0,Calle 146 #7F-54,LOS CEDROS,USAQUEN
3,400200,4.0,11,Si,76.0,3,2,1,2,Apartamento,0,24 hrs,No Tiene,Si,Si,442300000.0,KR 54 152A 35,EL PRADO,SUBA
4,270000,4.0,20,Si,105.0,4,2,1,0,Casa,1,24 hrs,No Tiene,Si,Si,387000000.0,CL 168 14 55,TOBERIN,USAQUEN


In [8]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   administración      585 non-null    int64  
 1   estrato             583 non-null    float64
 2   antiguedad          585 non-null    int64  
 3   remodelado          582 non-null    object 
 4   área                585 non-null    float64
 5   habitaciones        585 non-null    int64  
 6   baños               585 non-null    int64  
 7   garajes             585 non-null    int64  
 8   elevadores          585 non-null    int64  
 9   tipo_de_inmueble    585 non-null    object 
 10  deposito            585 non-null    int64  
 11  porteria            585 non-null    object 
 12  zona_de_lavanderia  585 non-null    object 
 13  gas                 555 non-null    object 
 14  parqueadero         585 non-null    object 
 15  precio              585 non-null    float64
 16  direccio

In [9]:
import pandas as pd



df_seguridad = pd.read_csv('Tasa_seguridad.csv')

# lo cual es la forma más "Pandas-friendly" y eficiente de un 'Left Merge'.

df_merged['Tasa_Homicidios_100k'] = df_merged['Localidad'].map(
    df_seguridad.set_index('Localidad')['Tasa_Homicidios_100k']
)
# 2. Mapear Tasa de Hurtos
df_merged['Tasa_Hurtos_100k'] = df_merged['Localidad'].map(
    df_seguridad.set_index('Localidad')['Tasa_Hurtos_100k']
)


In [10]:
df_merged.head()


,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,tipo_de_inmueble,...,porteria,zona_de_lavanderia,gas,parqueadero,precio,direccion,barrio_limpio,Localidad,Tasa_Homicidios_100k,Tasa_Hurtos_100k
0,532000,4.0,37,Si,86.0,1,1,1,1,Apartamento,...,24 hrs,No Tiene,Si,No,313900000.0,Avenida Carrera 3 # 59-65,PARDO RUBIO,CHAPINERO,5.2,1050.0
1,0,6.0,7,Si,77.0,1,2,2,1,Apartamento,...,24 hrs,No Tiene,Si,Si,440100000.0,Carrera 12 #101A-18,USAQUEN,USAQUEN,8.5,850.0
2,811893,4.0,14,Si,109.0,3,4,2,1,Apartamento,...,24 hrs,No Tiene,Si,Si,495000000.0,Calle 146 #7F-54,LOS CEDROS,USAQUEN,8.5,850.0
3,400200,4.0,11,Si,76.0,3,2,1,2,Apartamento,...,24 hrs,No Tiene,Si,Si,442300000.0,KR 54 152A 35,EL PRADO,SUBA,9.8,750.0
4,270000,4.0,20,Si,105.0,4,2,1,0,Casa,...,24 hrs,No Tiene,Si,Si,387000000.0,CL 168 14 55,TOBERIN,USAQUEN,8.5,850.0
